<a href="https://colab.research.google.com/github/tanvirnwu/PyTorch--Python/blob/main/Saving_and_Loading_PyTorch_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
!pip install torch_summary
from torchsummary import summary
from torch.optim import SGD
import time

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
class DataProcessing (Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).float().to(device)
        self.y = torch.tensor(y).float().to(device)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

In [4]:
X = np.random.randint(0, 10, size=(10, 2))

# Calculate the row-wise sum along axis=1
row_sums = np.sum(X, axis=1)

# Reshape the row_sums array to have a column shape
y = row_sums.reshape(-1, 1)
X, y

(array([[2, 5],
        [9, 0],
        [6, 5],
        [3, 5],
        [2, 5],
        [8, 3],
        [9, 9],
        [3, 0],
        [5, 9],
        [3, 0]]),
 array([[ 7],
        [ 9],
        [11],
        [ 8],
        [ 7],
        [11],
        [18],
        [ 3],
        [14],
        [ 3]]))

In [7]:
# Create an object of the DataProcessing class
dp = DataProcessing (X, y)
dp[1], dp[0]

((tensor([9., 0.], device='cuda:0'), tensor([9.], device='cuda:0')),
 (tensor([2., 5.], device='cuda:0'), tensor([7.], device='cuda:0')))

In [8]:
dloader = DataLoader(dp, batch_size=2, shuffle=True)

Creating a Sequential Model

In [10]:
model = nn.Sequential(
        nn.Linear(2, 6),
        nn.ReLU(),
        nn.Linear(6, 1)
        ).to(device)


In [11]:
summary(model, torch.zeros(1,2));

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 6]                   18
├─ReLU: 1-2                              [-1, 6]                   --
├─Linear: 1-3                            [-1, 1]                   7
Total params: 25
Trainable params: 25
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [12]:
#Define a loss function and optimizer
loss_func = nn.MSELoss()
optimizer = SGD(model.parameters(), lr = 0.001)

In [13]:
loss_history = []
start = time.time()

for xx in range(50):
  for X, y in dloader:
    optimizer.zero_grad()
    y_pred = model(X)
    loss_value = loss_func(y_pred, y)
    loss_value.backward()
    optimizer.step()
    loss_history.append(loss_value)

end = time.time()
print(f"Total Training Time = {end-start}")

Total Training Time = 0.3030283451080322


### Saving Model

In [14]:
save_path = "mymodel.pth"
torch.save(model.state_dict(), save_path)
!du -hsc {save_path} #Size of the model on disk

4.0K	mymodel.pth
4.0K	total


###Loading Model

In [15]:
load_path = "mymodel.pth"
model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

### Prediction Using the Loaded Model

In [17]:
X_test = np.random.randint(0, 10, size=(10, 2))
row_sums = np.sum(X_test, axis=1)

y_test = row_sums.reshape(-1, 1)
X_test = torch.tensor(X_test).float().to(device)
y_test = torch.tensor(y_test).float().to(device)
X_test, y_test

(tensor([[6., 8.],
         [2., 6.],
         [1., 5.],
         [7., 1.],
         [6., 5.],
         [5., 2.],
         [3., 4.],
         [6., 4.],
         [7., 4.],
         [6., 5.]], device='cuda:0'),
 tensor([[14.],
         [ 8.],
         [ 6.],
         [ 8.],
         [11.],
         [ 7.],
         [ 7.],
         [10.],
         [11.],
         [11.]], device='cuda:0'))

In [18]:
y_test_pred = model(X_test)
y_test_pred = y_test_pred.round().to(dtype=torch.int)
y_test_pred

tensor([[14],
        [ 8],
        [ 6],
        [ 8],
        [11],
        [ 7],
        [ 7],
        [10],
        [11],
        [11]], device='cuda:0', dtype=torch.int32)

### Accuracy

In [19]:
def calculate_accuracy(predicted_labels, true_labels):
    # Convert predicted and true labels to numpy arrays if they are tensors
    if torch.is_tensor(predicted_labels):
        predicted_labels = predicted_labels.cpu().numpy()
    if torch.is_tensor(true_labels):
        true_labels = true_labels.cpu().numpy()

    # Calculate accuracy
    num_correct = (predicted_labels == true_labels).sum()
    total_samples = len(true_labels)
    accuracy = num_correct / total_samples

    return accuracy

In [20]:
accuracy = calculate_accuracy(y_test_pred, y_test)
print(f"Accuracy: {accuracy*100}%")

Accuracy: 100.0%
